In [12]:
#importing necessary packages

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from datetime import datetime
import statsmodels.api as sm
import numpy as np 
import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_error
import imageio
import os
from statsmodels.graphics.tsaplots import plot_acf
import statsmodels.api as sm
from sklearn.model_selection import TimeSeriesSplit
from pycaret.time_series import TSForecastingExperiment


In [7]:
#função para tratar campo data
def parser(x):
    return pd.datetime.strptime('190'+x, '%Y-%m')

#Agora vamos importar nosso arquivo 
df = pd.read_excel('db_mt.xlsx', sheet_name='Sheet1')
print(df.head(10))

        year  value     liter
0 2017-12-31   190.2   301961
1 2018-12-31   192.9   545110
2 2019-12-31   194.8  1029190
3 2020-12-31   197.3  1997490
4 2021-12-31   200.8  2860097
5 2022-12-31   204.7  3209840
6 2023-12-31   208.2  4419242


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   year    7 non-null      datetime64[ns]
 1   value   7 non-null      float64       
 2   liter   7 non-null      int64         
dtypes: datetime64[ns](1), float64(1), int64(1)
memory usage: 296.0 bytes


In [14]:
df.drop(['liter'], axis = 1,inplace=True)

In [17]:
df.set_index('year',inplace=True)

In [18]:
df=df.sort_index(axis=0)
print(df.head(10))

            value 
year              
2017-12-31   190.2
2018-12-31   192.9
2019-12-31   194.8
2020-12-31   197.3
2021-12-31   200.8
2022-12-31   204.7
2023-12-31   208.2


fh: Optional[int or list or np.array], default = 1

The forecast horizon to be used for forecasting. Default is set to 1 i.e. forecast one point ahead. When integer is passed it means N continuous points in the future without any gap. If you want to forecast values with gaps, you must pass an array e.g. np.arange([13, 25]) will skip the first 12 future points and forecast from the 13th point till the 24th point ahead (note in numpy right value is inclusive and left is exclusive).

In [28]:
exp = TSForecastingExperiment()
exp.setup(data=df, fh=1, session_id=42, fig_kwargs = {"template": "simple_white","hoverinfo":"none"})

,Description,Value
0,session_id,42
1,Target,value
2,Approach,Univariate
3,Exogenous Variables,Not Present
4,Original data shape,"(7, 1)"
5,Transformed data shape,"(7, 1)"
6,Transformed train set shape,"(6, 1)"
7,Transformed test set shape,"(1, 1)"
8,Rows with missing values,0.0%
9,Fold Generator,ExpandingWindowSplitter


In [29]:
exp.plot_model(plot="ts",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})

## Checking Train-Test Split

In [30]:
exp.plot_model(plot="train_test_split",fig_kwargs = {"height":500,"width":1000,"template": "simple_white","hoverinfo":"none","big_data_threshold":10})


Comparing Models
The most basic assumption of time series forecasting is that past patterns continue into the future. Based on this assumption, in the short-term future forecast, since the environment in which the data is generated is similar to the present, the uncertainty is small and the forecast can be well-predicted. However, as we move towards long-term future forecasting, the possibility of the environment in which data is generated is likely to change, and unpredictable situations occur, and as errors continue to accumulate, uncertainty about the forecast data is bound to increase.

We compare the performance of various models and try to select the 4 models with the best performance.

## IMPORT Pycaret


In [ ]:
from pycaret.regression import *


Setting up the model is extremely easy

In [ ]:

reg = setup(data = train, 
             target = 'value',
             numeric_imputation = 'mean',
             categorical_features = ['year']  , 
            transformation = True, transform_target = True, 
                  combine_rare_levels = True, rare_level_threshold = 0.1,
                  remove_multicollinearity = True, multicollinearity_threshold = 0.95, 
             silent = True)

NameError: name 'train' is not defined

As a data scientist, I can't emphasize more on the usefulness of the function below - instead of pulling every single model, we just need one line to compare 20 different models! This is insane!

In [ ]:

# returns best models - takes a little time to run
top3 = compare_models(n_select = 3)

## Creating baseline model

In [ ]:
#we create a model using light gbm
lightgbm = create_model('lightgbm')

In [ ]:
tuned_lightgbm = tune_model(lightgbm) 

In [ ]:
plot_model(lightgbm, plot = 'error')

In [ ]:

plot_model(tuned_lightgbm, plot='feature') # looks like COVID-19 has played a huge role in sales

In [ ]:
predict_model(tuned_lightgbm);

In [ ]:
final_lightgbm = finalize_model(tuned_lightgbm)